In [ ]:
import sys
import os
from pathlib import Path
import glob
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import cv2

import torch
from torch import nn, Tensor
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
from tqdm import tqdm

from utils import get_abs_path, steer_to, lazy_states_contraction, is_feasible

In [ ]:
def cv2_imshow(img, title=None):
    img = np.concatenate((img[:,:,2:3], img[:,:,1:2], img[:,:,0:1]), axis=2)
    plt.imshow(img, cmap='gray')
    if title:
        plt.title(title)
    plt.show()


def display_image(image, title=None):
    image = image * 255
    image = image.permute(1, 2, 0).detach().numpy()
    cv2_imshow(image, title)

In [ ]:
project_path = get_abs_path(1)

model_dir = project_path + '/models/'
enet_filename = model_dir + 'enet.pt'
pnet_filename = model_dir + 'pnet.pt'

data_path = project_path + '/data/'
encoder_images_dir = data_path + 'encoder_images/'
train_images_dir = data_path + 'train/maps/'
train_paths_dir = data_path + 'train/planned_maps/'

valid_known_images_dir = data_path + 'valid/known_maps/'
valid_known_paths_dir = data_path + 'valid/planned_known_maps/'
valid_unknown_images_dir = data_path + 'valid/unknown_maps/'
valid_unknown_paths_dir = data_path + 'valid/planned_unknown_maps/'

test_known_images_dir = data_path + 'test/known_maps/'
test_known_paths_dir = data_path + 'test/planned_known_maps/'
test_unknown_images_dir = data_path + 'test/unknown_maps/'
test_unknown_paths_dir = data_path + 'test/planned_unknown_maps/'

train_enet = True
train_pnet = True

## **Create dataset for training encoder**

In [ ]:
class ImageDataset(Dataset):

    def __init__(self, images_paths):
        self.images_paths = images_paths

    def __len__(self):
        return len(self.images_paths)

    def __getitem__(self, idx):
        image_path = self.images_paths[idx]
        image = torch.Tensor(plt.imread(image_path))
        image = image.permute(2,1,0)
        image = image[2, :, :]
        return image, image

In [ ]:
encoder_images_paths = glob.glob(encoder_images_dir + '*.png')
train_encoder_paths, test_encoder_paths = train_test_split(encoder_images_paths, test_size=0.2)
valid_encoder_paths, test_encoder_paths = train_test_split(test_encoder_paths, test_size=0.5)

train_encoder_data = ImageDataset(train_encoder_paths)
valid_encoder_data = ImageDataset(valid_encoder_paths)
test_encoder_data = ImageDataset(test_encoder_paths)

batch_size = 5 #16
train_encoder_dataloader = DataLoader(train_encoder_data, batch_size=batch_size, shuffle=True, pin_memory=True)
valid_encoder_dataloader = DataLoader(valid_encoder_data, batch_size=batch_size, shuffle=True, pin_memory=True)
test_encoder_dataloader = DataLoader(test_encoder_data, batch_size=batch_size, shuffle=True, pin_memory=True)

In [ ]:
train_features, train_labels = next(iter(train_encoder_dataloader))
display_image(train_features[0].unsqueeze(0), title='Example map')

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {0} device'.format(device))

## **Create encoder model**

In [ ]:
class Encoder(nn.Module):
	def __init__(self, input_size):
		super(Encoder, self).__init__()
		self.flatten = nn.Flatten()
		self.dense1 = nn.Sequential(nn.Linear(input_size, 1512), nn.PReLU())
		self.dense2 = nn.Sequential(nn.Linear(1512, 556), nn.PReLU())
		self.dense3 = nn.Sequential(nn.Linear(556, 128), nn.PReLU())
		self.dense4 = nn.Linear(128, 32)

	def forward(self, x):
		x = self.flatten(x)
		x = self.dense1(x)
		x = self.dense2(x)
		x = self.dense3(x)
		x = self.dense4(x)
		return x


class Decoder(nn.Module):
	def __init__(self, output_size):
		super(Decoder, self).__init__()
		self.dense1 = nn.Sequential(nn.Linear(32, 128), nn.PReLU())
		self.dense2 = nn.Sequential(nn.Linear(128, 556), nn.PReLU())
		self.dense3 = nn.Sequential(nn.Linear(556, 1512), nn.PReLU())
		self.dense4 = nn.Sequential(nn.Linear(1512, output_size))
		self.unflatten = nn.Unflatten(1, (120, 120))

	def forward(self, x):
		x = self.dense1(x)
		x = self.dense2(x)
		x = self.dense3(x)
		x = self.dense4(x)
		x = self.unflatten(x)
		return x

In [ ]:
class ResUNet(nn.Module):
    def __init__(self, input_size):
        super(ResUNet, self).__init__()
        self.Encoder = Encoder(input_size)
        self.Decoder = Decoder(input_size)

    def forward(self, x):
        x = self.Encoder(x)
        x = self.Decoder(x)
        return x

Enet = ResUNet(14400).to(device)

## **Train Encoder**

In [ ]:
def contractive_mse_loss(model, output, target):

    mse_loss = nn.MSELoss()

    key = str(list(model.state_dict().keys())[-2])
    last_weigths = model.state_dict()[key]

    mse = mse_loss(output, target)
    lam = 1e-3
    contractive_loss = torch.sum(Variable(last_weigths)**2, dim=1).sum().mul_(lam)

    return mse + contractive_loss

In [ ]:
def train_encoder(model, optimizer, loss_fn, train_dataloader, valid_dataloader):

    for i in range(epochs):
        total_train_loss = 0
        total_valid_loss = 0

        it = 1
        model.train()
        with tqdm(train_dataloader, unit="batch", leave=False) as pbar:
            for X, y in pbar:
                pbar.set_description(f"Epoch {i + 1}/{epochs} training")
                X, y = X.to(device), y.to(device)

                pred = model(X)
                loss = loss_fn(model, pred, y)
                total_train_loss += loss

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                pbar.set_postfix(loss=total_train_loss.item() / it)
                it += 1

        it = 1
        with tqdm(valid_dataloader, unit="batch") as pbar:
            with torch.no_grad():
                for X, y in pbar:
                    pbar.set_description(f"Epoch {i + 1}/{epochs} validating")
                    X, y = X.to(device), y.to(device)

                    pred = model(X)
                    loss = loss_fn(model, pred, y)
                    total_valid_loss += loss
                    pbar.set_postfix(loss = total_train_loss.item() / len(train_dataloader), valid_loss = total_valid_loss.item() / it)
                    it += 1

In [ ]:
def test_encoder(model, loss_fn, dataloader):
    test_loss = 0
    with tqdm(dataloader, unit="batch") as pbar:
        with torch.no_grad():
            it = 1
            for X, y in pbar:
                pbar.set_description(f"Testing")
                X, y = X.to(device), y.to(device)
                pred = model(X)
                test_loss += loss_fn(model, pred, y).item()
                pbar.set_postfix(test_loss=test_loss / it)
                it += 1

In [ ]:
loss_fn = contractive_mse_loss
optimizer = torch.optim.Adagrad(Enet.parameters(), lr=1e-3)

if not os.path.exists(enet_filename) or train_enet:
    epochs = 60
    train_encoder(Enet, optimizer, loss_fn, train_encoder_dataloader, valid_encoder_dataloader)
    test_encoder(Enet, loss_fn, test_encoder_dataloader)
    torch.save(Enet.state_dict(), enet_filename)
else:
    Enet.load_state_dict(torch.load(enet_filename))
    Enet.eval()

## **Test Encoder**

In [ ]:
# Display Results
test_image, _ = next(iter(test_encoder_dataloader))
display_image(test_image[0].unsqueeze(0), 'Example map')

result = Enet(test_image.to(device))
display_image(result[0].cpu().unsqueeze(0), 'Map after reconstruction')

encoded = Enet.Encoder(test_image.to(device))[0]

## **Create path planning dataset**

In [ ]:
class PathDataset(Dataset):

    def __init__(self, images_dir, paths_dir, encoder):
        self.images_dir = images_dir
        self.paths_dir = paths_dir
        self.encoder = encoder

        images_paths = Path(paths_dir).glob('*/')
        self.sample_names = [x.name[0:-5] for x in images_paths if str(x).endswith('.json')]

    def __len__(self):
        return len(self.sample_names)


    def __getitem__(self, idx):
        image_name = self.images_dir + self.sample_names[idx][:-7] + '.png'
        path_name = self.paths_dir + self.sample_names[idx] + '.json'

        image = plt.imread(image_name)

        image = Tensor(image).to(device)
        image = image.permute(2,1,0)
        image = image[2, :, :]

        encoded_img = self.encoder(image.unsqueeze(0))[0].detach().cpu()

        file = open(path_name)
        data = Tensor(json.load(file))

        if len(data) == 2:
            pos_start = data[0] / image.shape[-1]
            pos_goal = data[1] / image.shape[-1]
            pos_pred = torch.round((data[0] + data[1]) / 2) / image.shape[-1]
        
        else:
            # Choose random consecutive points from planned path
            rand = np.random.randint(1, len(data) - 1)
            pos_start = data[rand - 1] / image.shape[-1]
            pos_goal = data[rand] / image.shape[-1]
            pos_pred = data[rand + 1] / image.shape[-1]

        input_data = torch.cat([torch.flatten(encoded_img), pos_start, pos_goal])
        input_data.requires_grad = True

        return input_data, pos_pred

    def get_image_path(self, idx):

        image_path = self.images_dir + self.sample_names[idx][:-7] + '.png'

        return image_path


train_dataset = PathDataset(train_images_dir, train_paths_dir, Enet.Encoder)
valid_dataset_known = PathDataset(valid_known_images_dir, valid_known_paths_dir, Enet.Encoder)
valid_dataset_unknown = PathDataset(valid_unknown_images_dir, valid_unknown_paths_dir, Enet.Encoder)
test_dataset_known = PathDataset(test_known_images_dir, test_known_paths_dir, Enet.Encoder)
test_dataset_unknown = PathDataset(test_unknown_images_dir, test_unknown_paths_dir, Enet.Encoder)

train_dataloader = DataLoader(train_dataset, batch_size=100, shuffle=True, pin_memory=True)
valid_dataloader_known = DataLoader(valid_dataset_known, batch_size=100, shuffle=True, pin_memory=True)
valid_dataloader_unknown = DataLoader(valid_dataset_unknown, batch_size=100, shuffle=True, pin_memory=True)
test_dataloader_known = DataLoader(test_dataset_known, batch_size=100, shuffle=True, pin_memory=True)
test_dataloader_unknown = DataLoader(test_dataset_unknown, batch_size=100, shuffle=True, pin_memory=True)

In [ ]:
class PlanningNetwork(nn.Module):
    def __init__(self, p=0.5):
        super(PlanningNetwork, self).__init__()

        enc_img_size = 36
        self.dense1 = nn.Sequential(nn.Linear(enc_img_size, 1280), nn.PReLU(), nn.Dropout(p))
        self.dense2 = nn.Sequential(nn.Linear(1280, 1024), nn.PReLU(), nn.Dropout(p))
        self.dense3 = nn.Sequential(nn.Linear(1024, 896), nn.PReLU(), nn.Dropout(p))
        self.dense4 = nn.Sequential(nn.Linear(896, 768), nn.PReLU(), nn.Dropout(p))
        self.dense5 = nn.Sequential(nn.Linear(768, 512), nn.PReLU(), nn.Dropout(p))
        self.dense6 = nn.Sequential(nn.Linear(512, 384), nn.PReLU(), nn.Dropout(p))
        self.dense7 = nn.Sequential(nn.Linear(384, 256), nn.PReLU(), nn.Dropout(p))
        self.dense8 = nn.Sequential(nn.Linear(256, 128), nn.PReLU(), nn.Dropout(p))
        self.dense9 = nn.Sequential(nn.Linear(128, 64), nn.PReLU(), nn.Dropout(p))
        self.dense10 = nn.Sequential(nn.Linear(64, 32), nn.PReLU())
        self.dense11 = nn.Sequential(nn.Linear(32, 2))
        
    def forward(self, x):

        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        x = self.dense5(x)
        x = self.dense6(x)
        x = self.dense7(x)
        x = self.dense8(x)
        x = self.dense9(x)
        x = self.dense10(x)
        x = self.dense11(x)

        return x

Pnet = PlanningNetwork().to(device)
print('Model parameters:', sum(p.numel() for p in Pnet.parameters() if p.requires_grad))

In [ ]:
class Pnet_loss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, output, target):

        loss = Tensor([0]).to(device)
        for i in range(len(target)):
            loss += torch.square(output[i, 0] - target[i, 0])
            loss += torch.square(output[i, 1] - target[i, 1])
        
        loss /= (target.shape[0] * 2)
        
        return loss

In [ ]:
def train_mpnet(model, optimizer, loss_fn, train_dataloader, valid_dataloader):

    for i in range(epochs):
        total_train_loss = 0
        total_valid_loss = 0

        it = 1
        model.train()
        with tqdm(train_dataloader, unit="batch", leave=False) as pbar:
            for X, y in pbar:
                pbar.set_description(f"Epoch {i + 1}/{epochs} training")
                X, y = X.to(device), y.to(device)

                pred = model(X)
                loss = loss_fn(pred, y)
                total_train_loss += loss

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                pbar.set_postfix(loss=total_train_loss / it)
                it += 1

        it = 1
        with tqdm(valid_dataloader, unit="batch") as pbar:
            with torch.no_grad():
                for X, y in pbar:
                    pbar.set_description(f"Epoch {i + 1}/{epochs} validating")
                    X, y = X.to(device), y.to(device)

                    pred = model(X)
                    loss = loss_fn(pred, y)
                    total_valid_loss += loss
                    pbar.set_postfix(loss = total_train_loss.item() / len(train_dataloader), valid_loss = total_valid_loss.item() / it)
                    it += 1

In [ ]:
def test_mpnet(model, loss_fn, test_dataloader):
    test_loss = 0
    with tqdm(test_dataloader, unit="batch") as pbar:
        with torch.no_grad():
            it = 1
            for X, y in pbar:
                pbar.set_description(f"Testing")
                X, y = X.to(device), y.to(device)
                pred = model(X)
                test_loss += loss_fn(pred, y).item()
                pbar.set_postfix(test_loss=test_loss / it)
                it += 1

In [ ]:
loss_fn = Pnet_loss().forward
optimizer = torch.optim.Adagrad(Pnet.parameters(), lr=1e-4)

if not os.path.exists(pnet_filename) or train_pnet:
    epochs = 5
    train_mpnet(Pnet, optimizer, loss_fn, train_dataloader, valid_dataloader_known)
    test_mpnet(Pnet, loss_fn, test_dataloader_known)
    torch.save(Pnet.state_dict(), pnet_filename)
else:
    Pnet.load_state_dict(torch.load(pnet_filename))
    Pnet.eval()



In [ ]:
# Display results
def display_results(model, dataset, idx):

    X, pos_pred = dataset.__getitem__(idx)

    print(X.to(device).unsqueeze(0))

    result = model(X.to(device).unsqueeze(0))

    pos_start = X[-4:-2]
    pos_goal = X[-2:]

    image_path = dataset.get_image_path(idx)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    points = [pos_start, result[0].detach().cpu(), pos_goal]
    colors = [(0, 255, 0), (255, 255, 0), (255, 0, 0)]

    for point, color in zip(points, colors):
        point = torch.round(point * 120)
        point = point.type(torch.uint8)
        
        image = cv2.rectangle(image, (point - 2).detach().numpy(), (point + 2).detach().numpy(), color, -1)

    for i in range(len(points) - 1):
        point1 = torch.round(points[i] * 120)
        point1 = point1.type(torch.uint8)

        point2 = torch.round(points[i + 1] * 120)
        point2 = point2.type(torch.uint8)

        image = cv2.line(image, point1.detach().numpy(), point2.detach().numpy(), (0, 255, 0), 1)

    plt.imshow(image)

display_results(Pnet, test_dataset_known, 0)

In [ ]:
# def Replaning(a, b):
#     pass

# N = 10

In [ ]:
# # Algorithm 1
# def MPNet(x_init, x_goal, x_obs):
#     Z = Enet.Encoder(x_obs)
#     Tau = NeuralPlanner(x_init, x_goal, Z)
#     if Tau:
#         Tau = lazy_states_contraction(Tau)

#         if is_feasible(Tau):
#             return Tau
#         else:
#             Tau_new = Replaning(Tau, Z)
#             Tau_new = lazy_states_contraction(Tau_new)
#             if is_feasible(Tau_new):
#                 return Tau_new
#     return 0

In [ ]:
# # Algorithm 2
# def NeuralPlanner(x_start, x_goal, Z):
#     Tau_a = [x_start]
#     Tau_b = [x_goal]
#     Tau = 0

#     for i in range(N):
#         x_new = Pnet(Z, Tau_a[-1], Tau_b[-1])
#         Tau_a.append(x_new)
#         connectable = steer_to(Tau_a[-1], Tau_b[-1])

#         if connectable:
#             Tau = np.concatenate((Tau_a, Tau_b))
#             return Tau

#         Tau_a, Tau_b = Tau_b, Tau_a

#     return 0